In [101]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time

In [ ]:
driver_options = webdriver.FirefoxOptions()
driver_options.add_argument("--headless")
driver = webdriver.Firefox(options=driver_options)

driver.get("https://draftlol.dawe.gg/Fng01vPk")
time.sleep(1)
html = driver.page_source
print(html)


In [ ]:
soup = BeautifulSoup(html, "html.parser")
print(soup.prettify())

In [ ]:
print(soup.find(attrs={"class": "roomPickColumn blue"}).get_text("|"))

In [ ]:
print(soup.find(attrs={"class": "roomPickColumn red"}).get_text("|"))

In [ ]:
print(soup.find(attrs={"class": "roomBanRow blue"}).prettify())

#banChampContainer -> Img (alt)

In [ ]:
[x['alt'] for x in soup.find(attrs={"class": "roomBanRow blue"}).find_all('img')]

In [ ]:
[x['alt'] for x in soup.find(attrs={"class": "roomBanRow red"}).find_all('img')]

In [ ]:
print("Blue team : ", soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundblue"}).previous_sibling.get_text())

In [ ]:
print("Red team : ", soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundred"}).previous_sibling.get_text())